In [10]:
import torch
import numpy as np

def three_channel_arr_to_shape(a, shape):
    pad_y_left, pad_y_right, pad_x_left, pad_x_right = get_pad_lengths(a,shape)
    return np.pad(a,((pad_y_left, pad_y_right), 
                     (pad_x_left, pad_x_right), (0,0)),
                  mode = 'constant')
def get_pad_lengths(a, shape):
    y_, x_ = shape
    y, x, c = a.shape
    y_pad = (y_-y)
    x_pad = (x_-x)
    pad_y_left = y_pad//2
    pad_y_right = y_pad//2 + y_pad%2
    pad_x_left = x_pad//2
    pad_x_right = x_pad//2 + x_pad%2
    return pad_y_left, pad_y_right, pad_x_left, pad_x_right

model = torch.jit.load('../models/megadetectorv5/md_v5a.0.0.torchscript')
  # set model parameters can go in inference basehandler subclass
model.conf = 0.10  # NMS confidence threshold
model.iou = 0.45  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.max_det = 1000  # maximum number of detections per image

In [11]:
import os
# TNC test images (local):
test_images_local = []
test_images_dir = os.path.abspath(os.path.join(os.path.abspath(''), '..', 'input'))
local_image_files = [
    'sample-img-empty.jpg',
    'sample-img.jpg',
    'sample-img-skunk-large.jpg',
    'sample-img-rodent.jpg',
    'sample-img-fox.jpg',
    'sample-img-fox-2.jpg',
]
for fil in local_image_files:
    test_images_local.append(os.path.join(test_images_dir, fil))

Wall time inference for single 2048x2048 image on cpu is 5.55 seconds

In [24]:
%%time
import skimage.io as skio
impath=test_images_local[2]
arr = skio.imread(impath)
padded_arr = three_channel_arr_to_shape(arr, (2048,2048))
im = torch.from_numpy(padded_arr)
im = torch.moveaxis(im,2,0).to("cpu").float()[None,...]
result = model(im)

CPU times: user 28.8 s, sys: 3.34 s, total: 32.1 s
Wall time: 5.55 s


wall time inference for single 2048x2048 image on gpu is .5 seconds

In [25]:
%%time
import yolov5
yolomodel = yolov5.load('../models/megadetectorv5/md_v5a.0.0.pt')
result_lst = yolomodel(impath)
result_lst.pred[0]

CPU times: user 412 ms, sys: 55.9 ms, total: 468 ms
Wall time: 468 ms


tensor([[9.37397e+02, 4.37510e+02, 1.34305e+03, 8.71688e+02, 9.30161e-01, 0.00000e+00]], device='cuda:0')

In [ ]:
# import yolov5

# # load pretrained model
yolomodel = yolov5.load('../models/megadetectorv5/md_v5a.0.0.pt')
# test_results = []
# for im in test_images_local:
#     result_lst = model(im)
#     test_results.append(result_lst)
#     result_lst.show()
#     result_lst.save(save_dir='results/')

torchscript all ims doesn't work

In [4]:
# import skimage.io as skio
# test_results = []
# for im in test_images_local:
#     im = torch.from_numpy(skio.imread(im))
#     result_lst = model(im)
#     test_results.append(result_lst)
#     result_lst.show()
#     result_lst.save(save_dir='results/')

In [ ]:
# predictions = result_lst.pred[0]
# boxes = predictions[:, :4] # x1, y1, x2, y2
# scores = predictions[:, 4]
# categories = predictions[:, 5]

In [24]:
# test_results[3].show()

### exporting the model for inference

clone the yolov5 repo (from [this commit](https://github.com/ultralytics/yolov5/blob/6dd6aea0866ba9115d38e2989f59cf1039b3c9d2/export.py) if master doesn't work). then:

In [ ]:
# !python export.py --weights ../animl-ml/models/megadetectorv5/md_v5a.0.0.pt --img 2500 --batch 1

In [ ]:
# https://sagemaker-examples.readthedocs.io/en/latest/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.html#When-should-I-extend-a-SageMaker-container?